In [3]:
# Load libraries
import numpy as np
import pandas as pd
from scipy import spatial
from random import sample
import copy
import re
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/prastutisingh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/prastutisingh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
# Top 50000 words from glove.42B.300d.zip were written to top_50000.txt.

# Load all data from massive embeddings file. You only need to do this once.
# Only adds nouns and proper nouns to 
embeddings_all = {}
with open("./top_50000.txt", 'r', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        tag = nltk.pos_tag([word])[0][1]
        if tag in ['NN', 'NNP'] and len(word) != 1:
            vector = np.asarray(values[1:], "float32")
            embeddings_all[word] = vector

# Load list of codenames words.
codenames_words = []
with open("./codenames_words.txt", 'r') as f:
    for line in f:
        codenames_words.append(line.strip())

In [5]:
# Function for choosing smaller sets of embeddings. 
def embeddings_size(embeddings_size, embeddings_all):
    #embeddings_size is the length of the subset of embeddings
    #embeddings_all is the embeddings from 50k top words
    embeddings = {}
    for x in list(embeddings_all)[0:embeddings_size]:
        embeddings[x] = embeddings_all[x]
    
    #adds words from codenames that don't make it in the top XX words
    for item in codenames_words:
        if item not in embeddings:
            try: 
                embeddings[item] = embeddings_all[item]
            except KeyError: 
                codenames_words.remove(item)
            
    return embeddings

In [6]:
#embeddings_1000 = embeddings_size(1000, embeddings_all)
#embeddings_5000 = embeddings_size(5000, embeddings_all)
#embeddings_10000 = embeddings_size(10000, embeddings_all)
embeddings_25000 = embeddings_size(25000, embeddings_all)
embeddings_50000 = embeddings_size(50000, embeddings_all)

In [17]:
# Helper Functions
def distance(embeddings, word, reference):
    return spatial.distance.cosine(embeddings[word], embeddings[reference])

def closest_words(embeddings, reference):
    return sorted(embeddings.keys(), key=lambda w: distance(embeddings, w, reference))

# Scoring function for generating clues based on sum of cosine distance
def goodness(embeddings, word, answers):
    if word in answers: return -999
    return sum([distance(embeddings, word, a) for a in answers])

# Chooses top candidates
def candidates(embeddings, answers, size=40):
    best = sorted(embeddings.keys(), key=lambda w: goodness(embeddings, w, answers))
    #res = [(str(i + 1), "{0:.2f}".format(minimax(embeddings, w, answers)), w) 
           #for i, w in enumerate(sorted(best[:250], key=lambda w: -1 * minimax(embeddings, w, answers))[:size])]
    
    return best

In [ ]:
# Functions to help with word + scores (not necessary for data collection)...
from itertools import zip_longest

def grouper(n, iterable, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)

def tabulate(data):
    data = list(grouper(10, data))
    return data

In [15]:
#from itertools import combinations 
    
# Generates a single example (good words, top ten clues)
def generate_single_example(codenames_words, embeddings):
    # Enforce threshold for 'good' words
#     while True: 
#         good = sample(codenames_words, 3)
#         combo_list = list(combinations(good, 2)) 
#         threshold_list = [distance(embeddings_all, item[0], item[1]) > 0.5 for item in combo_list]
#         if all(good): 
#             break
    good = sample(codenames_words, 3)
        
    clues = candidates(embeddings, good)
    
    # remove good words from clues
    for item in good: 
        if item in clues:
            clues.remove(item)
            
    print(good)
    for item in clues: 
        print(item)
        print(distance(embeddings_all, item, good[0]), distance(embeddings_all, item, good[1]), 
              distance(embeddings_all, item, good[2]))
                
    return good, clues[:10]

# Generates n examples and writes them to filename
def write_n_examples(codenames_words, embeddings, n, filename): 
    output_file = open(filename, 'w')
    
    for i in range(n): 
        good, clues = generate_single_example(codenames_words, embeddings)
        word_string = ', '.join(good + clues) + '\n'
        output_file.write(str(i) + '.' + word_string)
    
    output_file.close()
    
# write_n_examples(codenames_words, embeddings_1000, 10, 'test_examples.txt')

In [ ]:
# Not necessary yet...
def read_n_examples(filename):
    '''Reads filename of good, bad, clues examples into a three-dimensional array. 
    To access the 0th example: data[0]
    To access the good words (in list form) from the 1st example: data[1][0]
        good words list - 0
        bad words list - 1
        clue words list - 2
    To access the first clue word from the 1st example: data[1][2][0]'''
    file = open(filename, 'r')
    
    data = []

    for line in file: 
        full_list = line.strip('\n').split(',')
        good = full_list[:3]
        clues = full_list[9:]
        data.append([good, bad, clues])
        
    file.close()
    return data

In [16]:
write_n_examples(codenames_words, embeddings_50000, 1, 'examples_50000_vocab_nobadwords.txt')

['hood', 'lead', 'knife']
put
0.5560592412948608 0.4098125100135803 0.6275899708271027
blade
0.613227903842926 0.7082394957542419 0.3107225298881531
turn
0.5936126410961151 0.43128418922424316 0.6272522509098053
hand
0.644875556230545 0.5329477787017822 0.4819694757461548
cut
0.6198110580444336 0.5416776537895203 0.5014877021312714
pull
0.5546974837779999 0.5268367528915405 0.5826960206031799
side
0.507840484380722 0.5053401291370392 0.6586354374885559
head
0.5438684225082397 0.49127084016799927 0.6371569037437439
edge
0.6498264074325562 0.5275135338306427 0.4970549941062927
straight
0.6071048974990845 0.47242850065231323 0.614163339138031
hook
0.5183624923229218 0.6315774619579315 0.5592011511325836
way
0.6294119656085968 0.4103875160217285 0.6754920482635498
handle
0.6159647405147552 0.5898708403110504 0.511537492275238
sure
0.5701184272766113 0.483744740486145 0.665278822183609
tip
0.6322684586048126 0.5676049590110779 0.5198656916618347
need
0.6026749014854431 0.4363108277320862 0.

0.668133944272995 0.5204994678497314 0.7292332947254181
rod
0.6069402396678925 0.6346508264541626 0.6762942671775818
ground
0.6998589336872101 0.5614985823631287 0.6568011045455933
provide
0.6733539402484894 0.47782355546951294 0.7680543810129166
night
0.6677670776844025 0.5586311221122742 0.6946775317192078
reach
0.7301214635372162 0.4804990291595459 0.7107103765010834
couple
0.6661195755004883 0.5554090738296509 0.6998704969882965
pick
0.6934369802474976 0.5515826642513275 0.6765153408050537
fine
0.703936755657196 0.6034455001354218 0.6154899895191193
simple
0.6903558075428009 0.5543922483921051 0.6783491671085358
bone
0.6917688548564911 0.6494510471820831 0.582022488117218
hope
0.6738755404949188 0.4930076003074646 0.7567513138055801
finish
0.6824617087841034 0.5558004081249237 0.6860355138778687
beat
0.7191007733345032 0.5072272419929504 0.6979867815971375
scene
0.6412379741668701 0.6202337741851807 0.6632084548473358
become
0.7367613613605499 0.43332040309906006 0.7561451643705368

0.6091428697109222 0.6492851078510284 0.7462053298950195
game
0.7573602199554443 0.5427395403385162 0.7046921849250793
attention
0.6977090835571289 0.5220199227333069 0.7852744460105896
truck
0.608465850353241 0.6844387054443359 0.7121333479881287
strap
0.6023686826229095 0.7505817264318466 0.6522277295589447
seem
0.7069852352142334 0.5296925902366638 0.7689862102270126
cannot
0.7473531663417816 0.5229870080947876 0.7353412806987762
raise
0.7086035013198853 0.5539166927337646 0.7431894540786743
shirt
0.5746408998966217 0.7573382407426834 0.6738231778144836
dog
0.6929127275943756 0.6184210777282715 0.6946040093898773
price
0.6916579902172089 0.6357901990413666 0.6786392033100128
strip
0.6565245389938354 0.6390367746353149 0.7107616364955902
clark
0.6533642411231995 0.6204192340373993 0.7325826585292816
developed
0.7538722455501556 0.5261288583278656 0.7266457676887512
strike
0.7697219252586365 0.5945571660995483 0.6435500681400299
tube
0.6549945771694183 0.655241459608078 0.698032975196

stewart
0.6650498509407043 0.6265100538730621 0.7620483636856079
charge
0.7551412433385849 0.5343698263168335 0.7644464671611786
anderson
0.6544149816036224 0.5777539312839508 0.821800097823143
dave
0.7188983857631683 0.5780098140239716 0.7572890967130661
tape
0.7228760123252869 0.6638473272323608 0.6675067245960236
expert
0.7537255436182022 0.5596175789833069 0.740934818983078
pay
0.7192494869232178 0.5639821290969849 0.7712089121341705
tackle
0.7483073174953461 0.6173067390918732 0.68885138630867
pot
0.7001338601112366 0.7168131470680237 0.6378234326839447
fox
0.6525795161724091 0.6339503824710846 0.768416553735733
primary
0.7188858091831207 0.48762649297714233 0.8488168567419052
class
0.7264121174812317 0.6068592667579651 0.7222722172737122
decide
0.7536302357912064 0.5572622716426849 0.7447366118431091
west
0.6578312516212463 0.5882509052753448 0.8096125721931458
usa
0.7055599093437195 0.5930902361869812 0.7572091519832611
blind
0.6869821548461914 0.6464659571647644 0.7226880192756

0.8122624009847641 0.5266802310943604 0.7544636726379395
slash
0.6985352635383606 0.7322026491165161 0.6626753807067871
chose
0.7473013699054718 0.5833223462104797 0.7628254294395447
prefer
0.7338945865631104 0.6374022960662842 0.7222641706466675
lone
0.7402045130729675 0.6004878878593445 0.7528996616601944
sit
0.7217465341091156 0.607404351234436 0.7644824832677841
wont
0.670772910118103 0.6919981241226196 0.7309569120407104
specialist
0.7672067731618881 0.5542752146720886 0.77225461602211
tony
0.7633761316537857 0.5638286173343658 0.7665549218654633
robert
0.7088351845741272 0.6323859989643097 0.7526473253965378
motorcycle
0.6364267468452454 0.731703370809555 0.7257461845874786
cabinet
0.6768966615200043 0.7142229378223419 0.7027821838855743
sport
0.6475436687469482 0.7288584411144257 0.7177972793579102
drag
0.6496879160404205 0.6789695918560028 0.7656597346067429
nickel
0.7282263934612274 0.6039590835571289 0.7624696493148804
evil
0.7343372106552124 0.6155035495758057 0.744934439659

...

1.0173810590058565 0.9974585666786879 0.9908663975074887
nestor
1.04960123822093 0.9352387562394142 1.02095259539783
nathalie
1.0193581394851208 0.8795513585209846 1.1068837270140648
timestamp
0.9641774743795395 0.9897981267422438 1.0519099198281765
hotfix
1.0480430498719215 0.982114877551794 0.9758627098053694
tmg
1.0166512597352266 0.9693160932511091 1.0201140101999044
greensburg
1.0051315631717443 0.9728916492313147 1.0280935242772102
lordship
0.995805723592639 0.9697097167372704 1.040732629597187
nivea
0.9518555887043476 1.0347777307033539 1.0196236297488213
erisa
1.0452270321547985 0.91075599193573 1.050384495407343
nz$
0.9902715552598238 1.0194189585745335 0.9967335977125913
wmode
0.9855795009061694 1.0042405216954648 1.0166081190109253
aishwarya
0.9845638871192932 0.9212253838777542 1.1007276698946953
enchilada
1.0257173962891102 0.9948871722444892 0.9861274752765894
sat.
1.019038163125515 0.9657319523394108 1.0219784248620272
etihad
1.0502296388149261 0.8997438699007034 1.05683

giclée
0.9940538476221263 1.0667092949151993 0.9630331732332706
ohhhh
0.9678630493581295 1.0344428159296513 1.021515490487218
udaipur
1.0074638416990638 1.001788418740034 1.014572543092072
+91
1.0609215311706066 0.9463440366089344 1.016748271882534
memoriam
1.004791300278157 1.0143486307933927 1.004885339178145
bylaw
1.020176075398922 0.9581533186137676 1.0457248464226723
hermosa
1.0163470041006804 0.9798835664987564 1.0278497766703367
illusory
1.0378444828093052 0.9309780672192574 1.0553194433450699
become.com
0.9995721445593517 0.9868451645597816 1.0377554669976234
rehoboth
0.9180539846420288 1.0182553566992283 1.0880002155900002
i386
0.9767409432679415 1.0004072716401424 1.0474026910960674
lansdowne
1.0118933226913214 0.9630975760519505 1.0495659038424492
farsi
1.0567912608385086 0.9555456005036831 1.0122423898428679
thur
1.004831738770008 0.9729413446038961 1.046854231506586
listserv
1.0880814418196678 0.9683113060891628 0.9682723321020603
consular
1.0388109609484673 0.965132467448

0.9698097240179777 1.0112408101558685 1.0640588700771332
w1
1.0836159586906433 0.9795042313635349 0.9819976445287466
legislate
1.1187027916312218 0.8886361345648766 1.0379078462719917
points0
0.98021593503654 1.082036018371582 0.983071755617857
(530)
0.9905729591846466 1.0066467123106122 1.0482347197830677
strattera
0.9914803141728044 1.0074221002869308 1.0465611852705479
andorra
1.0289880465716124 0.9677006788551807 1.0488503240048885
wordnet
1.0476801432669163 0.9730551745742559 1.0250155627727509
rephrase
1.0628306716680527 0.9578211568295956 1.0251377280801535
quito
0.9593363367021084 0.9934288156218827 1.0930371657013893
entrevista
1.051989495754242 0.9595832377672195 1.0343715250492096
thanh
1.0393869541585445 0.9895256394520402 1.017168816179037
(415)
1.0250194855034351 0.9679398760199547 1.053129892796278
?!?!?!
1.0369557105004787 1.053450658917427 0.9557759910821915
(336)
0.9479201883077621 1.0100503731518984 1.0883324816823006
opa
1.0037795233074576 1.0332185477018356 1.00932

1.0322666577994823 1.0311863962560892 1.011323536746204
(919)
0.9905410315841436 1.029073566198349 1.0556075684726238
goji
1.03806771337986 0.9587810002267361 1.0785143673419952
commentaire
1.052959430962801 1.0060199005529284 1.0163892172276974
facepalm
1.0073923859745264 1.097911722958088 0.9702216759324074
immunol
0.9488011822104454 1.0772630795836449 1.0494640655815601
gardenweb
1.0139249572530389 1.0543281622231007 1.0073689073324203
descarga
1.0547867231070995 1.013076868839562 1.0077857575379312
expedia.com
1.0892423838377 1.007914662361145 0.9786263462156057
ardmore
1.033700704574585 1.012607586570084 1.0296421647071838
capitulo
1.007259770296514 1.0661021992564201 1.0026635380927473
babylonian
1.0279195681214333 0.9850219609215856 1.0631123036146164
(804)
0.969247467815876 1.0263269655406475 1.0805820897221565
instituto
1.006112058646977 1.0433383025228977 1.0267953127622604
alex.
1.0397810004651546 1.0349950157105923 1.0015191624406725
phy
1.0313822105526924 0.983653012663126

1.0442514009773731 1.0176993440836668 1.0761553049087524
docx
1.0391866452991962 0.9989245997276157 1.1005998775362968
datetime
1.0539217293262482 0.9994839462451637 1.0853786021471024
-857
0.9742431081831455 1.1840501576662064 0.9813192784786224
puta
1.064082846045494 1.030925216153264 1.045583214610815
encyclopædia
1.018273800611496 1.0989160686731339 1.0234958995133638
profilesend
1.0313317030668259 1.076925441622734 1.0331415012478828
(707)
1.0355013199150562 1.043258760124445 1.063172198832035
(949)
1.0587342269718647 1.0296334605664015 1.0539984442293644
nutr
1.0580609403550625 1.031368650496006 1.0549427941441536
system.your
1.1196410655975342 1.0413385108113289 0.9837928656488657
(717)
1.0176022462546825 1.0870987921953201 1.0401378385722637
clinicaltrials.gov
1.030080672353506 1.0116361295804381 1.1032428368926048
analyte
1.0514772981405258 0.9568589515984058 1.137168824672699
(504)
1.0099860178306699 1.0657014399766922 1.0701208263635635
ssf
1.0303732566535473 1.0842776671051

In [ ]:
#write_n_examples(codenames_words, embeddings_5000, 100, 'examples_5000_vocab_nobadwords.txt')
write_n_examples(codenames_words, embeddings_10000, 10, 'examples_10000_vocab_nobadwords.txt')